In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
import os
from PIL import Image
import json
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, recall_score, accuracy_score
from torch.utils.data import Dataset
from torchvision import transforms
import numpy as np
from sklearn.metrics import recall_score
from tqdm import tqdm

In [4]:
import torch
from torch.utils.data import Dataset
import json
from PIL import Image
import os

data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

class SpoofDataset(Dataset):
    def __init__(self, data_dir, label_file, transform=None):
        """
        Args:
            data_dir: Directory with all the images, separated into 'live' and 'spoof' subdirectories.
            label_file: Path to JSON file with labels, where label 0 indicates 'live' and others indicate 'spoof'.
            transform: Optional transform.
        """
        self.data_dir = data_dir
        self.transform = transform
        with open(label_file, 'r') as f:
            self.labels = json.load(f)

        self.img_paths = []
        self.img_labels = []
        for img_name, label in self.labels.items():
            # Correct the file extension from .png to .jpg
            #img_name = img_name.replace('.png', '.jpg')
            subfolder = 'live' if label == 0 else 'spoof'
            full_path = os.path.join(self.data_dir, subfolder, img_name)
            if os.path.exists(full_path):
                self.img_paths.append(full_path)
                self.img_labels.append(label)

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.img_labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label


In [5]:
data_dir = 'preprocessed_data'
label_file = 'test_label.json'

dataset = SpoofDataset(data_dir=data_dir, label_file=label_file, transform=data_transforms)

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import transforms
import numpy as np

class Generator(nn.Module):
    def __init__(self, z_dim, num_classes, img_shape):
        super(Generator, self).__init__()
        self.label_emb = nn.Embedding(num_classes, num_classes)
        self.gen = nn.Sequential(
            nn.Linear(z_dim + num_classes, 128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(128, 256),
            nn.BatchNorm1d(256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 512),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, np.prod(img_shape)),
            nn.Tanh()
        )
        self.img_shape = img_shape

    def forward(self, noise, labels):
        label_input = self.label_emb(labels)
        gen_input = torch.cat((noise, label_input), -1)
        img = self.gen(gen_input)
        img = img.view(img.size(0), *self.img_shape)
        return img

class Discriminator(nn.Module):
    def __init__(self, num_classes, img_shape):
        super(Discriminator, self).__init__()
        self.label_embedding = nn.Embedding(num_classes, np.prod(img_shape))
        self.dis = nn.Sequential(
            nn.Linear(np.prod(img_shape) + np.prod(img_shape), 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
            nn.Sigmoid(),
        )

    def forward(self, img, labels):
        label_input = self.label_embedding(labels).view(img.size(0), 1, *img.size()[1:])
        d_in = torch.cat((img.view(img.size(0), -1), label_input.view(img.size(0), -1)), -1)
        validity = self.dis(d_in)
        return validity

# Hyperparameters (adjust these based on your dataset)
z_dim = 100
num_classes = 5  # 'live' or 'spoof'
img_shape = (3, 64, 64)  # Assuming 64x64 RGB images

# Initialize generator and discriminator
generator = Generator(z_dim, num_classes, img_shape)
discriminator = Discriminator(num_classes, img_shape)

# Optimizers
optimizer_G = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizer_D = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))

# Loss function
adversarial_loss = nn.BCELoss()

# Device setting
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
generator.to(device)
discriminator.to(device)
adversarial_loss.to(device)


BCELoss()

In [14]:
import os
from torchvision.utils import save_image

# Create the directory for saving images if it does not exist
image_dir = "images"
if not os.path.exists(image_dir):
    os.makedirs(image_dir)


In [17]:
data_dir = 'preprocessed_data'
label_file = 'test_label.json'
# Training Hyperparameters
n_epochs = 10  # Number of epochs to train for
batch_size = 64  # Batch size
sample_interval = 400  # Interval to save generated images for inspection

# Configure DataLoader
# Assuming 'SpoofDataset' is your dataset class as defined earlier
transform = transforms.Compose([
    transforms.Resize(img_shape[1:]),  # Resize images to the expected dimensions
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])  # Normalize images
])
dataset = SpoofDataset(data_dir=data_dir, label_file=label_file, transform=transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Initialize labels for real and fake images
real_labels = torch.ones(batch_size, 1, device=device, dtype=torch.float)
fake_labels = torch.zeros(batch_size, 1, device=device, dtype=torch.float)

for epoch in range(n_epochs):
    for i, (imgs, labels) in enumerate(dataloader):
        
        # Adapting batch size for the last batch
        current_batch_size = imgs.size(0)
        real_labels = torch.ones(current_batch_size, 1, device=device, dtype=torch.float)
        fake_labels = torch.zeros(current_batch_size, 1, device=device, dtype=torch.float)
        
        # Configure input
        real_imgs = imgs.to(device)
        labels = labels.to(device)

        # ---------------------
        #  Train Discriminator
        # ---------------------
        optimizer_D.zero_grad()

        # Loss for real images
        real_loss = adversarial_loss(discriminator(real_imgs, labels), real_labels)

        # Loss for fake images
        noise = torch.randn(current_batch_size, z_dim, device=device)
        # Generate a batch of images
        gen_imgs = generator(noise, labels)
        fake_loss = adversarial_loss(discriminator(gen_imgs.detach(), labels), fake_labels)

        # Total discriminator loss
        d_loss = (real_loss + fake_loss) / 2
        d_loss.backward()
        optimizer_D.step()

        # -----------------
        #  Train Generator
        # -----------------
        optimizer_G.zero_grad()

        # Loss measures generator's ability to fool the discriminator
        # Train on fake images
        gen_imgs = generator(noise, labels)
        g_loss = adversarial_loss(discriminator(gen_imgs, labels), real_labels)

        g_loss.backward()
        optimizer_G.step()

        print(f"[Epoch {epoch}/{n_epochs}] [Batch {i}/{len(dataloader)}] [D loss: {d_loss.item()}] [G loss: {g_loss.item()}]")

        # Save Images for inspection, every 'sample_interval' batches
        if i % sample_interval == 0:
            save_image(gen_imgs.data[:25], f"images/{epoch}_{i}.png", nrow=5, normalize=True)


[Epoch 0/10] [Batch 0/1050] [D loss: 0.5579312443733215] [G loss: 0.8595078587532043]
[Epoch 0/10] [Batch 1/1050] [D loss: 0.6370548605918884] [G loss: 1.537828803062439]
[Epoch 0/10] [Batch 2/1050] [D loss: 0.5093957781791687] [G loss: 1.1733627319335938]
[Epoch 0/10] [Batch 3/1050] [D loss: 0.563868522644043] [G loss: 1.345081090927124]
[Epoch 0/10] [Batch 4/1050] [D loss: 0.48135608434677124] [G loss: 1.4969500303268433]
[Epoch 0/10] [Batch 5/1050] [D loss: 0.5291777849197388] [G loss: 1.1028435230255127]
[Epoch 0/10] [Batch 6/1050] [D loss: 0.4410218894481659] [G loss: 1.652512788772583]
[Epoch 0/10] [Batch 7/1050] [D loss: 0.45206624269485474] [G loss: 1.248803973197937]
[Epoch 0/10] [Batch 8/1050] [D loss: 0.5270118713378906] [G loss: 1.3041338920593262]
[Epoch 0/10] [Batch 9/1050] [D loss: 0.5159283876419067] [G loss: 1.4670897722244263]
[Epoch 0/10] [Batch 10/1050] [D loss: 0.48696303367614746] [G loss: 1.1821415424346924]
[Epoch 0/10] [Batch 11/1050] [D loss: 0.437038451433181

[Epoch 0/10] [Batch 96/1050] [D loss: 0.4778115749359131] [G loss: 1.4818819761276245]
[Epoch 0/10] [Batch 97/1050] [D loss: 0.5695652961730957] [G loss: 1.3014715909957886]
[Epoch 0/10] [Batch 98/1050] [D loss: 0.5032435655593872] [G loss: 1.4093257188796997]
[Epoch 0/10] [Batch 99/1050] [D loss: 0.566078782081604] [G loss: 1.1231034994125366]
[Epoch 0/10] [Batch 100/1050] [D loss: 0.5077205300331116] [G loss: 1.2722508907318115]
[Epoch 0/10] [Batch 101/1050] [D loss: 0.514655351638794] [G loss: 1.519584059715271]
[Epoch 0/10] [Batch 102/1050] [D loss: 0.6343938708305359] [G loss: 0.840470016002655]
[Epoch 0/10] [Batch 103/1050] [D loss: 0.5198261737823486] [G loss: 1.429272174835205]
[Epoch 0/10] [Batch 104/1050] [D loss: 0.4829096794128418] [G loss: 1.502060055732727]
[Epoch 0/10] [Batch 105/1050] [D loss: 0.5333775877952576] [G loss: 1.1304770708084106]
[Epoch 0/10] [Batch 106/1050] [D loss: 0.5334492921829224] [G loss: 1.0518207550048828]
[Epoch 0/10] [Batch 107/1050] [D loss: 0.5

[Epoch 0/10] [Batch 191/1050] [D loss: 0.48213258385658264] [G loss: 1.0592353343963623]
[Epoch 0/10] [Batch 192/1050] [D loss: 0.4771401286125183] [G loss: 1.7154344320297241]
[Epoch 0/10] [Batch 193/1050] [D loss: 0.43262195587158203] [G loss: 1.8040367364883423]
[Epoch 0/10] [Batch 194/1050] [D loss: 0.5146048069000244] [G loss: 1.1212139129638672]
[Epoch 0/10] [Batch 195/1050] [D loss: 0.451839804649353] [G loss: 1.7310742139816284]
[Epoch 0/10] [Batch 196/1050] [D loss: 0.5899251699447632] [G loss: 1.252570390701294]
[Epoch 0/10] [Batch 197/1050] [D loss: 0.5626111030578613] [G loss: 1.1839982271194458]
[Epoch 0/10] [Batch 198/1050] [D loss: 0.5246757864952087] [G loss: 1.4888074398040771]
[Epoch 0/10] [Batch 199/1050] [D loss: 0.4535984992980957] [G loss: 1.6535922288894653]
[Epoch 0/10] [Batch 200/1050] [D loss: 0.5725370645523071] [G loss: 0.8594357371330261]
[Epoch 0/10] [Batch 201/1050] [D loss: 0.5709055662155151] [G loss: 1.5898151397705078]
[Epoch 0/10] [Batch 202/1050] [D

[Epoch 0/10] [Batch 285/1050] [D loss: 0.46365582942962646] [G loss: 1.4365564584732056]
[Epoch 0/10] [Batch 286/1050] [D loss: 0.5230903625488281] [G loss: 1.1968104839324951]
[Epoch 0/10] [Batch 287/1050] [D loss: 0.4944760799407959] [G loss: 1.5207223892211914]
[Epoch 0/10] [Batch 288/1050] [D loss: 0.48687809705734253] [G loss: 1.2634847164154053]
[Epoch 0/10] [Batch 289/1050] [D loss: 0.546204686164856] [G loss: 0.8899935483932495]
[Epoch 0/10] [Batch 290/1050] [D loss: 0.4950890839099884] [G loss: 1.8472634553909302]
[Epoch 0/10] [Batch 291/1050] [D loss: 0.48868024349212646] [G loss: 1.335594654083252]
[Epoch 0/10] [Batch 292/1050] [D loss: 0.44742345809936523] [G loss: 1.2698112726211548]
[Epoch 0/10] [Batch 293/1050] [D loss: 0.44973552227020264] [G loss: 1.3408877849578857]
[Epoch 0/10] [Batch 294/1050] [D loss: 0.4426003694534302] [G loss: 1.5120604038238525]
[Epoch 0/10] [Batch 295/1050] [D loss: 0.5107534527778625] [G loss: 0.9409828186035156]
[Epoch 0/10] [Batch 296/1050]

[Epoch 0/10] [Batch 380/1050] [D loss: 0.5163688659667969] [G loss: 1.2175666093826294]
[Epoch 0/10] [Batch 381/1050] [D loss: 0.5031709671020508] [G loss: 1.579267144203186]
[Epoch 0/10] [Batch 382/1050] [D loss: 0.43055713176727295] [G loss: 1.5052255392074585]
[Epoch 0/10] [Batch 383/1050] [D loss: 0.42254915833473206] [G loss: 1.4237202405929565]
[Epoch 0/10] [Batch 384/1050] [D loss: 0.5725275874137878] [G loss: 0.9552644491195679]
[Epoch 0/10] [Batch 385/1050] [D loss: 0.5300891995429993] [G loss: 1.5482343435287476]
[Epoch 0/10] [Batch 386/1050] [D loss: 0.40235283970832825] [G loss: 1.8455232381820679]
[Epoch 0/10] [Batch 387/1050] [D loss: 0.4924182891845703] [G loss: 0.8649986386299133]
[Epoch 0/10] [Batch 388/1050] [D loss: 0.5096293687820435] [G loss: 2.1823956966400146]
[Epoch 0/10] [Batch 389/1050] [D loss: 0.45841825008392334] [G loss: 1.314182996749878]
[Epoch 0/10] [Batch 390/1050] [D loss: 0.5333203673362732] [G loss: 1.3457624912261963]
[Epoch 0/10] [Batch 391/1050] 

...

[Epoch 9/10] [Batch 664/1050] [D loss: 0.5375295281410217] [G loss: 1.120377540588379]
[Epoch 9/10] [Batch 665/1050] [D loss: 0.6183242201805115] [G loss: 0.9179195165634155]
[Epoch 9/10] [Batch 666/1050] [D loss: 0.6103833913803101] [G loss: 1.1813297271728516]
[Epoch 9/10] [Batch 667/1050] [D loss: 0.5323241949081421] [G loss: 1.1468515396118164]
[Epoch 9/10] [Batch 668/1050] [D loss: 0.5986385941505432] [G loss: 1.2629711627960205]
[Epoch 9/10] [Batch 669/1050] [D loss: 0.6059662699699402] [G loss: 1.3600867986679077]
[Epoch 9/10] [Batch 670/1050] [D loss: 0.5680334568023682] [G loss: 0.9709233045578003]
[Epoch 9/10] [Batch 671/1050] [D loss: 0.6149373054504395] [G loss: 1.3744863271713257]
[Epoch 9/10] [Batch 672/1050] [D loss: 0.6449652910232544] [G loss: 1.2936551570892334]
[Epoch 9/10] [Batch 673/1050] [D loss: 0.5445749759674072] [G loss: 1.055058240890503]
[Epoch 9/10] [Batch 674/1050] [D loss: 0.597573459148407] [G loss: 1.4685355424880981]
[Epoch 9/10] [Batch 675/1050] [D lo

[Epoch 9/10] [Batch 758/1050] [D loss: 0.5561324954032898] [G loss: 1.0883675813674927]
[Epoch 9/10] [Batch 759/1050] [D loss: 0.6065274477005005] [G loss: 1.0242668390274048]
[Epoch 9/10] [Batch 760/1050] [D loss: 0.49550268054008484] [G loss: 1.3378835916519165]
[Epoch 9/10] [Batch 761/1050] [D loss: 0.5142999291419983] [G loss: 1.3121471405029297]
[Epoch 9/10] [Batch 762/1050] [D loss: 0.5999101400375366] [G loss: 1.0522253513336182]
[Epoch 9/10] [Batch 763/1050] [D loss: 0.5276360511779785] [G loss: 1.1705844402313232]
[Epoch 9/10] [Batch 764/1050] [D loss: 0.613036036491394] [G loss: 1.2721318006515503]
[Epoch 9/10] [Batch 765/1050] [D loss: 0.5860912203788757] [G loss: 0.8497747778892517]
[Epoch 9/10] [Batch 766/1050] [D loss: 0.5913863182067871] [G loss: 1.6306779384613037]
[Epoch 9/10] [Batch 767/1050] [D loss: 0.6388891935348511] [G loss: 0.8920665979385376]
[Epoch 9/10] [Batch 768/1050] [D loss: 0.5860466957092285] [G loss: 1.1885037422180176]
[Epoch 9/10] [Batch 769/1050] [D

[Epoch 9/10] [Batch 853/1050] [D loss: 0.5638747215270996] [G loss: 1.0382695198059082]
[Epoch 9/10] [Batch 854/1050] [D loss: 0.5588502883911133] [G loss: 1.3195374011993408]
[Epoch 9/10] [Batch 855/1050] [D loss: 0.590535581111908] [G loss: 0.9323931336402893]
[Epoch 9/10] [Batch 856/1050] [D loss: 0.5508858561515808] [G loss: 1.3077945709228516]
[Epoch 9/10] [Batch 857/1050] [D loss: 0.5366050004959106] [G loss: 1.1134228706359863]
[Epoch 9/10] [Batch 858/1050] [D loss: 0.59528648853302] [G loss: 0.9693247675895691]
[Epoch 9/10] [Batch 859/1050] [D loss: 0.5386896729469299] [G loss: 1.7332584857940674]
[Epoch 9/10] [Batch 860/1050] [D loss: 0.6427974700927734] [G loss: 0.8213140964508057]
[Epoch 9/10] [Batch 861/1050] [D loss: 0.5651024580001831] [G loss: 1.1750801801681519]
[Epoch 9/10] [Batch 862/1050] [D loss: 0.5347321033477783] [G loss: 1.452674150466919]
[Epoch 9/10] [Batch 863/1050] [D loss: 0.5907927751541138] [G loss: 0.9480417966842651]
[Epoch 9/10] [Batch 864/1050] [D los

[Epoch 9/10] [Batch 948/1050] [D loss: 0.5886988639831543] [G loss: 1.1344605684280396]
[Epoch 9/10] [Batch 949/1050] [D loss: 0.5536823272705078] [G loss: 1.285966157913208]
[Epoch 9/10] [Batch 950/1050] [D loss: 0.5795333981513977] [G loss: 0.9936143755912781]
[Epoch 9/10] [Batch 951/1050] [D loss: 0.5561641454696655] [G loss: 1.170527458190918]
[Epoch 9/10] [Batch 952/1050] [D loss: 0.5360260605812073] [G loss: 1.1798361539840698]
[Epoch 9/10] [Batch 953/1050] [D loss: 0.6012706756591797] [G loss: 1.1279094219207764]
[Epoch 9/10] [Batch 954/1050] [D loss: 0.6083153486251831] [G loss: 0.9929205775260925]
[Epoch 9/10] [Batch 955/1050] [D loss: 0.5817587971687317] [G loss: 1.4782813787460327]
[Epoch 9/10] [Batch 956/1050] [D loss: 0.5397343635559082] [G loss: 1.1014095544815063]
[Epoch 9/10] [Batch 957/1050] [D loss: 0.5856072902679443] [G loss: 1.0334163904190063]
[Epoch 9/10] [Batch 958/1050] [D loss: 0.5923137664794922] [G loss: 1.3755799531936646]
[Epoch 9/10] [Batch 959/1050] [D l

[Epoch 9/10] [Batch 1042/1050] [D loss: 0.5338324308395386] [G loss: 1.2742674350738525]
[Epoch 9/10] [Batch 1043/1050] [D loss: 0.5719529390335083] [G loss: 1.0278265476226807]
[Epoch 9/10] [Batch 1044/1050] [D loss: 0.5653617978096008] [G loss: 1.3983253240585327]
[Epoch 9/10] [Batch 1045/1050] [D loss: 0.5187874436378479] [G loss: 1.0815937519073486]
[Epoch 9/10] [Batch 1046/1050] [D loss: 0.5385491847991943] [G loss: 1.3083317279815674]
[Epoch 9/10] [Batch 1047/1050] [D loss: 0.5413405895233154] [G loss: 1.1704751253128052]
[Epoch 9/10] [Batch 1048/1050] [D loss: 0.5502976179122925] [G loss: 1.2784786224365234]
[Epoch 9/10] [Batch 1049/1050] [D loss: 0.5445481538772583] [G loss: 1.3032646179199219]
